<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/text-to-code/text-to-sql.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Text-to-SQL

This notebook demonstrates how to use the Gretel Navigator SDK to create a synthetic data generation pipeline for SQL code examples. We'll build a system that generates SQL code based on natural language instructions, with varying complexity levels and industry focuses.

In this notebook, we'll demonstrate how to add columns using a short-hand syntax where you do not need to use concrete Python Types. 

In [ ]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U gretel_client 

In [ ]:
from gretel_client.navigator_client import Gretel

## 📘 Setting Up the Data Designer

First, we'll initialize the Data Designer with appropriate system instructions.

In [ ]:
# Initialize Gretel client and Data Designer
gretel = Gretel(api_key="prompt", endpoint="https://api.dev.gretel.ai")
aidd = gretel.data_designer.new(
    model_suite="apache-2.0"  # Use apache-2.0 or llama-3.x based on your licensing needs
)

## 🌱 Define Categorical Seed Columns

We'll set up our seed columns for industry sectors, code complexity, and instruction types. These will help generate diverse and relevant SQL examples.

In [ ]:
# Add industry sector categories
aidd.add_column(
    name="industry_sector",
    type="category",
    params={
        "values": ["Healthcare", "Finance", "Technology"],
        "description": "The industry sector for the SQL example"
    }
)

# Add topic as a subcategory of industry_sector
aidd.add_column(
    name="topic",
    type="subcategory",
    params={
        "category": "industry_sector",
        "values": {
            "Healthcare": [
                "Electronic Health Records (EHR) Systems",
                "Telemedicine Platforms", 
                "AI-Powered Diagnostic Tools"
            ],
            "Finance": [
                "Fraud Detection Software",
                "Automated Trading Systems",
                "Personal Finance Apps"
            ],
            "Technology": [
                "Cloud Computing Platforms",
                "Artificial Intelligence and Machine Learning Platforms",
                "DevOps and CI/CD Tools"
            ]
        }
    }
)

# Add SQL complexity with subcategory for SQL concepts
aidd.add_column(
    name="sql_complexity",
    type="category",
    params={
        "values": ["Beginner", "Intermediate", "Advanced"],
        "description": "The complexity level of the SQL code"
    }
)

# Add SQL concept as a subcategory of sql_complexity
aidd.add_column(
    name="sql_concept",
    type="subcategory",
    params={
        "category": "sql_complexity",
        "values": {
            "Beginner": [
                "Basic SELECT Statements",
                "WHERE Clauses",
                "Basic JOINs",
                "INSERT, UPDATE, DELETE"
            ],
            "Intermediate": [
                "Aggregation Functions",
                "Multiple JOINs",
                "Subqueries",
                "Views"
            ],
            "Advanced": [
                "Window Functions",
                "Common Table Expressions (CTEs)",
                "Stored Procedures",
                "Query Optimization"
            ]
        }
    }
)

# Add SQL task types
aidd.add_column(
    name="sql_task_type",
    type="category",
    params={
        "values": [
            "Data Retrieval",
            "Data Manipulation",
            "Analytics and Reporting",
            "Data Transformation"
        ],
        "description": "The type of SQL task being performed"
    }
)

# Add instruction phrases
aidd.add_column(
    name="instruction_phrase",
    type="category",
    params={
        "values": [
            "Write an SQL query that",
            "Create an SQL statement to",
            "Develop an SQL query to",
            "Can you write SQL that",
            "Formulate an SQL query that"
        ],
        "description": "Starting phrase for the SQL instruction"
    }
)

## ✨ Define Generated Data Columns

Now we'll set up the columns that will be generated by the LLMs, including the instruction, database context, and SQL implementation.

In [ ]:
from gretel_client.data_designer import params as P
# Generate instruction for the SQL query
aidd.add_column(
    name="sql_prompt",
    type="llm-text",
    system_prompt="You are an expert at generating clear and specific SQL tasks.",
    prompt="""\
Generate an instruction to create SQL code that solves a specific problem. 
Each instruction should begin with one of the following phrases: {{instruction_phrase}}.

Important Guidelines:
* Industry Relevance: Ensure the instruction pertains to the {{industry_sector}} sector and {{topic}} topic.
* SQL Complexity: Tailor the instruction to the {{sql_complexity}} level. Utilize relevant {{sql_concept}} where appropriate to match the complexity level.
* Task Type: The instruction should involve a {{sql_task_type}} task.
* Clarity and Specificity: Make the problem statement clear and unambiguous. Provide sufficient context to understand the requirements without being overly verbose.
* Response Formatting: Do not include any markers such as ### Response ### in the instruction.
"""
)

# Generate database context
aidd.add_column(
    name="sql_context",
    type="llm-code",
    output_format=P.CodeLang.SQL_ANSI, # Specify CodeLang.SQL_ANSI to ensure the code is structured as valid SQL
    system_prompt="You are an expert SQL database designer who creates clean, efficient, and well-structured database schemas.",
    prompt="""\
Generate the SQL for creating database tables that would be relevant for the following instruction:
Instruction: {{sql_prompt}}

Important Guidelines:
* Relevance: Ensure all tables are directly related to the {{industry_sector}} sector and {{topic}} topic.
* Completeness: Include all essential columns with appropriate data types, primary/foreign keys, and necessary constraints.
* Realism: Use realistic table structures typical for the specified industry.
* Executable SQL: Provide complete CREATE TABLE statements that can be run without modification.
* Consistency: Use consistent naming conventions (e.g., snake_case for table and column names).
* Sample Data: Include INSERT statements with sample data that makes sense for the tables (at least 5-10 rows per table).
"""
)

# Generate the SQL code
aidd.add_column(
    name="sql",
    type="llm-code",
    output_format=P.CodeLang.SQL_ANSI, # Specify CodeLang.SQL_ANSI to ensure the code is structured as valid SQL
    system_prompt="You are an expert SQL programmer who writes clean, efficient, and well-structured queries.",
    prompt="""\
Write SQL code for the following instruction based on the provided database context:
Instruction: {{sql_prompt}}

Database Context:
{{sql_context}}

Important Guidelines:
* Code Quality: Your SQL should be clean, complete, self-contained and accurate.
* Code Validity: Please ensure that your SQL code is executable and does not contain any errors.
* Context: Base your query on the provided database context. Only reference tables and columns that exist in the context.
* Complexity & Concepts: The SQL should be written at a {{sql_complexity}} level, making use of concepts such as {{sql_concept}}.
* Task Type: Ensure your solution implements the appropriate {{sql_task_type}} operation.
* Comments: Include brief comments explaining the key parts of your query.
"""
)

## 🔍 Add Validation and Evaluation

Let's add post-processing steps to validate the generated code and evaluate the text-to-SQL conversion.

In [ ]:
# Add validators and evaluators
from gretel_client.data_designer.judge_rubrics import TEXT_TO_SQL_LLM_JUDGE_PROMPT_TEMPLATE, SQL_RUBRICS

aidd.add_column(name="sql_validity_result", type="code-validation", code_lang=P.CodeLang.SQL_ANSI, target_column="sql")
aidd.add_column(name="sql_judge_result", type="llm-judge", prompt=TEXT_TO_SQL_LLM_JUDGE_PROMPT_TEMPLATE, rubrics=SQL_RUBRICS)

## 👀 Generate Preview Dataset

Let's generate a preview to see some data.

In [ ]:
aidd.with_evaluation_report()

In [ ]:
# Generate a preview
preview = aidd.preview()

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [ ]:
preview.display_sample_record()

## 🚀 Generate Full Dataset

If you're satisfied with the preview, you can generate a larger dataset using a batch workflow.

In [ ]:
# Submit batch job
workflow_run = aidd.create(
    num_records=100,
    name="text_to_sql_examples"
)

workflow_run.wait_until_done()

print("\nGenerated dataset shape:", workflow_run.dataset.df.shape)

# Download evaluation report
workflow_run.report.download("report.html", format="html")